In [1]:
data = [1, 2, 3, 4, 5]

print(data[:-1])

[1, 2, 3, 4]


In [1]:
import xarray_beam as xbeam
import xarray

ds, _ = xbeam.open_zarr('gcs://weatherbench2/datasets/era5/1959-2023_01_10-full_37-1h-512x256_equiangular_conservative.zarr')
print(ds.nbytes / (10**9))

84453.77358148


In [3]:
import pandas as pd
import numpy as np

start_date = pd.to_datetime('2021-01-01')
end_date = pd.to_datetime('2021-01-03')

lat_min, lat_max = (32, 39.3)
lon_min, lon_max = (124, 131.4)

# 해당 범위에 속하는 위도와 경도의 인덱스 d
lat_indices = np.where((ds.latitude >= lat_min) & (ds.latitude <= lat_max))[0]
lon_indices = np.where((ds.longitude >= lon_min) & (ds.longitude <= lon_max))[0]

source_dataset = ds.sel(time=slice(start_date, end_date))

source_dataset = source_dataset.isel(latitude=lat_indices, longitude=lon_indices)

In [4]:
print(source_dataset)

<xarray.Dataset>
Dimensions:                                           (time: 49, longitude: 10,
                                                       latitude: 10, level: 37)
Coordinates:
  * latitude                                          (latitude) float64 32.7...
  * level                                             (level) int64 1 2 ... 1000
  * longitude                                         (longitude) float64 124...
  * time                                              (time) datetime64[ns] 2...
Data variables: (12/48)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 ...
    10m_v_component_of_wind                           (time, longitude, latitude) float32 ...
    2m_dewpoint_temperature                           (time, longitude, latitude) float32 ...
    2m_temperature                                    (time, longitude, latitude) float32 ...
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 ..

In [ ]:
dar = ds_kor['2m_temperature']
dar = dar.sel(level=100)

data = dar.to_numpy()
print(data.shape)

In [ ]:
print(data)

In [ ]:
import pandas as pd
dar = ds['2m_temperature']
i2 = pd.to_datetime(str(2018) + '-03-01T00:00:00.000000000') 
dar = dar.sel(time=i2)
print(dar)

In [ ]:
ds_kor = dar.isel(latitude=lat_indices, longitude=lon_indices)
print(ds_kor.latitude)
print(ds_kor.longitude)


In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = (6,6) #차트 사이즈
from mpl_toolkits.basemap import Basemap, cm
m = Basemap(llcrnrlon=min(ds_kor.longitude),llcrnrlat=min(ds_kor.latitude),urcrnrlon=max(ds_kor.longitude),urcrnrlat=max(ds_kor.latitude), resolution='i',projection='cyl',lon_0=(max(ds_kor.longitude)-min(ds_kor.longitude)) / 2,lat_0=(max(ds_kor.latitude)-min(ds_kor.latitude)) / 2)
ds_kor.plot()
m.drawcoastlines()